# Inference of Trained Retinanet model 
### Import Libraries

In [40]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from numpy import genfromtxt
import time
import json
import copy
import math

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
#setup_gpu(gpu)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
file_location = 'list_of_img_in_val_set4.csv'
place_to_store_results = 'Evaluations/'
path_img_folder = '../../../03 Data/Simple Dataset/'
accept_BB_threshold = 0.7

def draw_vector(img, x, y, box):
    angle = np.arctan2(y,x)
    angle = angle/2
    center_x = (box[0]+box[2])/2
    center_y = (box[1]+box[3])/2
    pt1_x = -25*np.cos(angle)+center_x
    pt1_y = -25*np.sin(angle)+center_y
    pt2_x = 25*np.cos(angle)+center_x
    pt2_y = 25*np.sin(angle)+center_y
    cv2.line(img, (int(pt1_x),int(pt1_y)), (int(pt2_x), int(pt2_y)), color = (0, 255,255), thickness = 2, lineType = 2)
    return
    

def convert_rect_to_point(rect):
    x = rect[0]
    y = rect[1]
    w = rect[2]
    h = rect[3]
    x1 = x
    y1 = y
    x2 = x + w
    y2 = y + h
    
    return (x1, y1, x2, y2)

def read_annotations_from_json(img_name):
    with open(path_img_folder + img_name.strip('.png') + '.json') as json_file:
        data = json.load(json_file)
        shapes_list = data['shapes']
        annotation_list = []
        for annotation in shapes_list: 
            rect = cv2.boundingRect(np.float32(annotation['points']))
            annotation_list.append(convert_rect_to_point(rect))
    return annotation_list; 


def max_val(val1, val2):
    if val1 > val2:
        return val1
    else:
        return val2
    
def min_val(val1, val2):
    if val1 > val2:
        return val2
    else:
        return val1
    
    
def filter_bounding_boxes(bounding_boxes, scores):
    list_of_BB = []
    for box, score in zip(bounding_boxes, scores):
        if score < accept_BB_threshold: 
            break
        list_of_BB.append(box)
    return list_of_BB


def area(BB):
    width = abs(BB[0] - BB[2])
    height = abs(BB[1] - BB[3])
    return height * width

def intersection(BB1, BB2):
    # find coordiantes of intersection rectangle 
    (x1, y1, x2, y2) = BB1
    (x3, y3, x4, y4) = BB2
    if x1 > x4 or x3 > x2 or y3 > y2 or y1 > y4:
        return 0
    x5 = max_val(x1, x3);
    y5 = max_val(y1, y3);
    x6 = min_val(x2, x4);
    y6 = min_val(y2, y4);  
    BB = (x5, y5, x6, y6)
    # TODO: check if no intersection exists
    return area(BB)
    
def union(BB1, BB2): 
    return area(BB1) + area(BB2) - intersection(BB1, BB2)

def evaluate_predictions(annotations, preds, IOU_thres):
    predictions = copy.deepcopy(preds)
    false_neg = 0
    for annotation in annotations: 
        #annotation = convert_rect_to_point(anno)
        max_IOU = 0
        for i in range(len(predictions)): 
            #print('Annotation: ', annotation)
            #print('pred candidate: ', predictions[i])
            if intersection(annotation, predictions[i]) > 0: #convert_rect_to_point only for tests
                IOU = intersection(annotation, predictions[i])/union(annotation, predictions[i])
                #print(IOU)  
                if(IOU > max_IOU):
                    max_IOU_index = i
                    max_IOU = IOU
        if max_IOU > IOU_thres:
            # remove prediction from list
            predictions.pop(max_IOU_index)
        else:
            false_neg = false_neg + 1
    
    false_pos = len(predictions)
    true_pos = len(preds) - len(predictions)
    return (false_neg, false_pos, true_pos)        

### Import Data and Model 

In [13]:
# import data
file = open(file_location)
file_paths = list(file)  

print(file_paths)

# import model
model_path = os.path.join('..','..','Create-CSV-dataset', 'snapshots', 'resnet50_csv_25.h5');

model = models.load_model(model_path, backbone_name='resnet50');

# If model is not converted to inference model, use line below: 
model = models.convert_model(model)


labels_to_names = {0: 'Brick'};

#labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

['colorIMG_75.png\n', 'colorIMG_32.png\n', 'colorIMG_114.png\n', 'colorIMG_95.png\n', 'colorIMG_43.png\n', 'colorIMG_67.png\n', 'colorIMG_24.png\n', 'colorIMG_12.png\n', 'colorIMG_159.png\n', 'colorIMG_51.png\n', 'colorIMG_103.png\n', 'colorIMG_96.png\n', 'colorIMG_50.png\n', 'colorIMG_179.png\n', 'colorIMG_94.png\n', 'colorIMG_88.png\n', 'colorIMG_77.png\n', 'colorIMG_108.png\n', 'colorIMG_37.png\n', 'colorIMG_152.png\n', 'colorIMG_9.png\n', 'colorIMG_45.png\n', 'colorIMG_99.png\n', 'colorIMG_111.png\n', 'colorIMG_201.png\n', 'colorIMG_78.png\n', 'colorIMG_47.png\n', 'colorIMG_15.png\n', 'colorIMG_46.png\n', 'colorIMG_183.png\n', 'colorIMG_62.png\n', 'colorIMG_136.png\n', 'colorIMG_116.png\n', 'colorIMG_147.png\n', 'colorIMG_53.png\n', 'colorIMG_107.png\n', 'colorIMG_132.png\n', 'colorIMG_44.png\n', 'colorIMG_190.png\n', 'colorIMG_55.png\n']
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 

### Infer Images

In [8]:
IOU_threshold = 0.5
total_false_positive = 0
total_true_positive = 0
total_false_negative = 0

for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))

    # create copy to draw on 
    draw = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLhttp://cocodataset.org/#detection-evalOR_BGR2RGB)
    
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image 
    prediction = model.predict_on_batch(np.expand_dims(image, axis=0))
    print(prediction)

[array([[[723.7614 , 531.59454, 899.5797 , 601.94244],
        [731.13513, 485.03534, 902.47626, 561.0911 ],
        [561.30896, 572.009  , 735.4083 , 641.36676],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999988 ,  0.9999963 ,  0.99999225,  0.9999919 ,  0.99998844,
         0.99998796,  0.999972  ,  0.9999677 ,  0.9999541 ,  0.9999316 ,
         0.99983096,  0.9997533 ,  0.99961287,  0.99951077,  0.999345  ,
         0.99799025,  0.82584566,  0.13161987,  0.08591333,  0.059542  ,
         0.05417114, -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[542.9977 , 362.23373, 715.0029 , 424.20596],
        [548.17346, 323.2968 , 718.0477 , 381.91245],
        [538.70984, 405.29547, 709.16205, 463.6329 ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999869 ,  0.99998033,  0.99997306,  0.9999728 ,  0.9999529 ,
         0.9999373 ,  0.9998869 ,  0.9998461 ,  0.99971884,  0.9996854 ,
         0.9995683 ,  0.9984707 ,  0.99845874,  0.9979826 ,  0.9976267 ,
         0.99657154,  0.7569056 ,  0.11667649,  0.06558638, -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[547.15607, 426.4079 , 719.40344, 484.26666],
        [545.5645 , 509.50464, 718.0773 , 568.8839 ],
        [549.15125, 384.05536, 720.63196, 442.63943],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999913 ,  0.9999912 ,  0.9999807 ,  0.9999691 ,  0.99996865,
         0.9999578 ,  0.9998853 ,  0.9998772 ,  0.9998178 ,  0.99975187,
         0.99964905,  0.99953055,  0.9994215 ,  0.99923074,  0.99921095,
         0.99856037,  0.9900087 ,  0.3244634 ,  0.18597694,  0.08274084,
         0.08074459,  0.06668098,  0.06034503, -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[713.2016 , 287.60217, 888.6606 , 348.30228],
        [714.37463, 239.19963, 893.3884 , 309.66226],
        [706.1166 , 373.367  , 880.227  , 429.55118],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.99999297,  0.9999924 ,  0.9999912 ,  0.99999046,  0.9999665 ,
         0.9999548 ,  0.99994385,  0.9998988 ,  0.99988854,  0.9998418 ,
         0.99983776,  0.9998068 ,  0.99980444,  0.9996897 ,  0.99967635,
         0.99915123,  0.98841965,  0.6845478 ,  0.50445   ,  0.09314442,
         0.06775916,  0.06039956, -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[784.134  , 361.88895, 919.68335, 519.2324 ],
        [688.6343 , 286.4007 , 824.57153, 440.0749 ],
        [688.5423 , 492.07364, 816.6081 , 655.1075 ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9998381 ,  0.9997366 ,  0.99964297,  0.99961346,  0.99954   ,
         0.9994461 ,  0.9994443 ,  0.9993081 ,  0.998889  ,  0.9987129 ,
         0.9978806 ,  0.9926226 ,  0.99227494,  0.2658654 ,  0.2106809 ,
         0.1180533 ,  0.11358332,  0.08857659,  0.08600561,  0.08283988,
         0.08144057,  0.08066373,  0.07885463,  0.07821531,  0.06789687,
         0.06575416,  0.06305291,  0.05918606,  0.05598214,  0.05588672,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

...

[array([[[704.74316, 481.56497, 878.72534, 553.4423 ],
        [550.6317 , 377.60654, 725.0945 , 440.75076],
        [721.1784 , 359.11295, 896.6283 , 429.67645],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999976 ,  0.99999523,  0.99999475,  0.99999404,  0.99999297,
         0.99999285,  0.9999925 ,  0.99999213,  0.9999913 ,  0.99999094,
         0.99998784,  0.99998736,  0.9999796 ,  0.99996614,  0.99990547,
         0.99989545,  0.12291862,  0.12049431,  0.10091919,  0.09159493,
         0.06210692,  0.06199032,  0.05187796, -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[695.2531 , 526.994  , 870.9816 , 607.3275 ],
        [698.4856 , 486.36166, 873.1498 , 548.6851 ],
        [703.7168 , 447.89288, 873.3782 , 502.3685 ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999989 ,  0.9999937 ,  0.9999558 ,  0.9998467 ,  0.9997844 ,
         0.9995516 ,  0.99945396,  0.9994361 ,  0.9989586 ,  0.99888545,
         0.99855834,  0.99831414,  0.9977272 ,  0.99766964,  0.9963834 ,
         0.9942133 ,  0.8603284 ,  0.56326747,  0.07000477,  0.06182392,
         0.05847943, -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[707.83887, 563.26984, 884.1546 , 634.108  ],
        [711.5282 , 520.05316, 887.18945, 580.4286 ],
        [734.80615, 267.3269 , 909.93115, 326.1746 ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999994 ,  0.9999958 ,  0.99999356,  0.9999845 ,  0.9999844 ,
         0.99995625,  0.99995613,  0.9999505 ,  0.9999348 ,  0.99992263,
         0.99992204,  0.99990106,  0.999851  ,  0.999495  ,  0.9988194 ,
         0.998572  ,  0.86389303,  0.42103615,  0.13443589,  0.12884802,
         0.10195074,  0.05259947,  0.05117835, -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[542.2321 , 241.66237, 591.7387 , 406.74432],
        [674.6875 , 229.96318, 720.5768 , 396.6007 ],
        [756.1705 , 238.59422, 806.1445 , 405.443  ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9982114 ,  0.9969483 ,  0.9962643 ,  0.99586415,  0.99503136,
         0.9950187 ,  0.9949462 ,  0.9947978 ,  0.9944449 ,  0.9934882 ,
         0.99340725,  0.99337655,  0.9921704 ,  0.9906879 ,  0.99039125,
         0.9825052 ,  0.86532086,  0.534769  ,  0.26244614,  0.11880186,
         0.09814619,  0.08617199,  0.07115135,  0.06773693,  0.0627666 ,
         0.05867655,  0.05751726,  0.05610042,  0.05609544,  0.05573651,
         0.05529067,  0.05456698,  0.05398749,  0.05044866,  0.05005373,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[526.89655, 507.414  , 700.1037 , 595.2933 ],
        [507.19873, 426.72565, 680.87885, 515.46893],
        [483.42502, 344.66257, 654.47473, 436.2623 ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999918 ,  0.9999877 ,  0.999964  ,  0.9999614 ,  0.99996006,
         0.9999342 ,  0.99991727,  0.99991095,  0.9999057 ,  0.9998915 ,
         0.9998865 ,  0.9998512 ,  0.99981016,  0.9997887 ,  0.9995802 ,
         0.9995214 ,  0.2655741 ,  0.15598586,  0.13554935,  0.1340878 ,
         0.12061307,  0.08492724,  0.08024929,  0.06328828,  0.06267174,
         0.05952827,  0.05943262,  0.05917458,  0.05192806,  0.05113348,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

In [48]:
IOU_threshold = 0.5
total_false_positive = 0
total_true_positive = 0
total_false_negative = 0
alpha = 0.5

for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))

    # create copy to draw on 
    draw = image.copy()
    draw2 = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLhttp://cocodataset.org/#detection-evalOR_BGR2RGB)
    
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image 
    boxes, scores, labels, angles = model.predict_on_batch(np.expand_dims(image, axis=0))
    
    # correct for image scale
    boxes /= scale
    
    # print detections
    for box, score, label, angles in zip(boxes[0], scores[0], labels[0], angles[0]):
        # scores are sorted so we can break
        if score < accept_BB_threshold:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)
        #cv2.addWeighted(draw, alpha, draw2, 1 - alpha,0, draw)
        

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
        draw_vector(draw, angles[0], angles[1], box)
    
    # load json file containing annotations
    annotations = read_annotations_from_json(path.strip('\n'))
    
    # remove bounding boxes with a classification score below accept_BB_threshold. 
    predictions = filter_bounding_boxes(boxes[0], scores[0])
    
    #calculate intersection and union of two bounding boxes
    (false_neg, false_pos, true_pos) = evaluate_predictions(annotations, predictions, IOU_threshold)
    #results = evaluate_predictions(annotations, annotations)
    total_false_negative = total_false_negative + false_neg
    total_false_positive = total_false_positive + false_pos
    total_true_positive  = total_true_positive  + true_pos
    
    # store image
    cv2.imwrite(place_to_store_results + path.strip('\n'), draw)

# Calculate precision, recall and F1 score 
precision = total_true_positive / (total_true_positive + total_false_positive)
recall = total_true_positive / (total_true_positive + total_false_negative)
f1 = 2 * (precision * recall) / (precision + recall) 

# print result
print('Precision: ', precision)
print('Recall: ', recall)
print('f1: ', f1)

Precision:  0.9434523809523809
Recall:  0.990625
f1:  0.9664634146341462
